# Indicadores de Trazabilidad 



## Importación de datos, librerias

In [1]:
# Importa librerias necesarias para el análisis

import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# Importa datos de población regional y crea DF


df_poblacion = pd.read_csv('data/aysen_population.csv') #Archivo ubicado dentro de la carpeta data

df_poblacion.head(2)

,Unnamed: 0,orden,nombre_region,codigo_region,nombre_provincia,codigo_provincia,nombre_comuna,codigo_comuna,edad_hombres,edad_mujeres,total
0,0,7172,Región de Aysén del General Carlos Ibáñez del ...,11,Coyhaique,111,Coyhaique,11101,28763,29055,57818
1,1,7194,Región de Aysén del General Carlos Ibáñez del ...,11,Coyhaique,111,Lago Verde,11102,494,358,852


In [3]:
# Importa datos de comunas corregidas por establecimiento de salud

df_establecimiento = pd.read_excel('data/establecimientos_salud_dict.xlsx') #Archivo ubicado dentro de la carpeta data


df_establecimiento.head()

,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi
0,Actividades gestionadas por la Dirección del S...,Coyhaique,Coyhaique
1,Centro Clínico Militar Coyhaique,Coyhaique,Coyhaique
2,Centro Comunitario de Salud Familiar Villa Ma...,Aysen,Aysen
3,Centro Comunitario de Salud Familiar Alejandro...,Coyhaique,Coyhaique
4,Centro Comunitario de Salud Familiar Puerto Ch...,Aysen,Aysen


In [4]:
# Importa datos desde la base de datos recivida via FTP
## PRIMERO: Guardar archivo dentro de la carpeta data
## SEGUNDO: No cambiar nombre ni formato del archivo recibido
## TERCERO: RECORDAR CAMBIAR FECHA EN EL ARCHIVO A IMPORTAR A CONTINUACION, i.e: 20210829 para este domingo 29 de Agosto 2021

df =  pd.read_csv('data/20210831_Región de Aysén del General Carlos Ibáñez del Campo_notificaciones_0600.csv', 
                   sep=';', # CSV con separador ''~''
                   low_memory=False
                  )

In [5]:
df_trazabilidad =  pd.read_csv('data/20210831_Región de Aysén del General Carlos Ibáñez del Campo_Seguimiento.csv', 
                   sep=';', # CSV con separador ''~''
                   low_memory=False
                  )

##  limpieza y ordenamiento 

In [6]:
# Une datos con diccionario comunal

dff = df.merge(df_establecimiento, how='left')

In [7]:
# Integra datos poblacionales

df_merged = dff.merge(df_poblacion.drop(columns= ['Unnamed: 0','orden','nombre_region', 'codigo_region', 'nombre_provincia','codigo_provincia']), left_on='comuna_seremi_xi', right_on='nombre_comuna', how='left' )

In [8]:
# Filtra datos obtenidos solo para pacientes con residencia en la Región de Aysén

data_aysen = df_merged[df_merged['region_residencia'] == 'Región de Aysén del General Carlos Ibáñez del Campo']

data_aysen.head(1)

,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,seremi,...,ningun_factor_riesgo,test_antigeno_sars_positivo,tac_torax_covid,comuna_seremi_xi,provincia_seremi_xi,nombre_comuna,codigo_comuna,edad_hombres,edad_mujeres,total
0,100166,50166,50,Coronavirus (COVID-19),11,2020-03-12,DESCARTADA,Hospital Regional (Coihaique),Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,NaN,NaN,NaN,Coyhaique,Coyhaique,Coyhaique,11101.0,28763.0,29055.0,57818.0


In [9]:
data_aysen_filtered = data_aysen[(data_aysen['estado_caso']!='No validada')&(
                          data_aysen['vigente_no_eliminado']=='t')&(
                          data_aysen['seremi']=='SEREMI De Aisén del General Carlos Ibañez del Campo')
                       ].rename(columns={'edad_hombres':'pop_hombres',
                                         'edad_mujeres':'pop_mujeres',
                                         'total':'pop_total'})

data_aysen_filtered.sample(1)

,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,seremi,...,ningun_factor_riesgo,test_antigeno_sars_positivo,tac_torax_covid,comuna_seremi_xi,provincia_seremi_xi,nombre_comuna,codigo_comuna,pop_hombres,pop_mujeres,pop_total
1155,600834,550834,50,Coronavirus (COVID-19),22,2020-05-29,DESCARTADA,Hospital de Puerto Aysén,Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,f,NaN,NaN,Aysen,Aysen,Aysen,11201.0,12719.0,11240.0,23959.0


In [10]:
# Transforma Fechas a formato DateTime 

data_aysen_filtered['fecha_notificacion'] = pd.to_datetime(data_aysen_filtered['fecha_notificacion'], errors = 'coerce') 

data_aysen_filtered['fecha_toma_muestra_1'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_1'], errors = 'coerce')

data_aysen_filtered['fecha_toma_muestra_2'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_2'], errors = 'coerce')

data_aysen_filtered['fecha_toma_muestra_3'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_3'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_1'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_1'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_2'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_2'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_3'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_3'], errors = 'coerce')

In [11]:
# Muestra variables presente en DB en formato DF

s=pd.DataFrame(data_aysen_filtered.columns)

s[s[0]=='tipo_seguimiento']


,0


In [12]:
# Selecciona variables de importancia

data_aysen_clean =  data_aysen_filtered[[
                                'numero_folio',
                                'comuna_residencia',
                                'fecha_notificacion',
                                'semana_epidemiologica',
                                'etapa_clinica',
                                'tipo_caso_busqueda',
                                'establecimiento_salud',
                                'resultado_pcr_1',
                                'resultado_pcr_2',
                                'resultado_pcr_3',
                                'fecha_toma_muestra_1',
                                'fecha_toma_muestra_2',
                                'fecha_toma_muestra_3',
                                'fecha_resultado_pcr_1',
                                'fecha_resultado_pcr_2',
                                'fecha_resultado_pcr_3',
                                'estado_paciente',
                                'lugar_reposo',
                                'estado_caso',
                                'presentacion_clinica',
                                'contacto_caso_sospechoso',
                                'motivo_examen',
                                'hospitalizacion',
                                'trabajador_salud',
                                'caso_cerrado',
                                'fecha_primeros_sintomas',
                                'fecha_diagnostico',
                                'comuna_seremi_xi',
                                'provincia_seremi_xi',
                                'pop_hombres',
                                'pop_mujeres',
                                'pop_total'
                               ]]

In [13]:
# Agrega 53 semanas a las semanas de 2021 y crea semana_epidemiologica_21

data_aysen_clean1 = data_aysen_clean[data_aysen_clean['fecha_notificacion']<=pd.to_datetime('2021-01-02')]
data_aysen_clean2 = data_aysen_clean[data_aysen_clean['fecha_notificacion']>=pd.to_datetime('2021-01-03')]


data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53

data_aysen_clean_21 = data_aysen_clean1.append(data_aysen_clean2, ignore_index=True)


<ipython-input-13-d2f259f8832e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
<ipython-input-13-d2f259f8832e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53


In [14]:
data_aysen_clean.sample(10)

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,tipo_caso_busqueda,establecimiento_salud,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,...,hospitalizacion,trabajador_salud,caso_cerrado,fecha_primeros_sintomas,fecha_diagnostico,comuna_seremi_xi,provincia_seremi_xi,pop_hombres,pop_mujeres,pop_total
143095,14632517,Coyhaique,2021-07-06,27,DESCARTADA,Búsqueda activa,Actividades gestionadas por la Dirección del S...,NEGATIVO,NaN,NaN,...,NaN,DESCONOCIDO,f,NaN,2021-07-07,Coyhaique,Coyhaique,28763.0,29055.0,57818.0
108060,12090849,Aisén,2021-05-11,19,DESCARTADA,Atención médica,Hospital de Puerto Aysén,NEGATIVO,NaN,NaN,...,NaN,NO,f,2021-05-09,2021-05-11,Aysen,Aysen,12719.0,11240.0,23959.0
150270,15180826,Coyhaique,2021-07-21,29,DESCARTADA,Búsqueda activa,Consultorio Víctor Domingo Silva,NaN,NEGATIVO,NaN,...,NaN,NO,f,NaN,2021-07-22,Coyhaique,Coyhaique,28763.0,29055.0,57818.0
53519,7071215,Cisnes,2021-01-18,3,DESCARTADA,Búsqueda activa,Hospital Dr. Jorge Ibar (Cisnes),NaN,NaN,NaN,...,NaN,NO,f,NaN,2021-01-19,Cisnes,Aysen,3922.0,2595.0,6517.0
11208,2770932,Coyhaique,2020-09-16,38,DESCARTADA,Búsqueda activa,Consultorio Alejandro Gutiérrez,NEGATIVO,NaN,NaN,...,NaN,NO,f,NaN,2020-09-18,Coyhaique,Coyhaique,28763.0,29055.0,57818.0
147934,14977594,Coyhaique,2021-07-15,28,DESCARTADA,Atención médica,Hospital Regional (Coihaique),NaN,NaN,NaN,...,NaN,SI,f,NaN,2021-07-15,Coyhaique,Coyhaique,28763.0,29055.0,57818.0
117896,12763082,Coyhaique,2021-05-26,21,PROBABLE,Atención médica,Actividades gestionadas por la Dirección del S...,NEGATIVO,NaN,NaN,...,NaN,SI,f,2021-05-19,2021-05-26,Coyhaique,Coyhaique,28763.0,29055.0,57818.0
6938,2070977,Coyhaique,2020-08-23,35,DESCARTADA,Atención médica,Hospital Regional (Coihaique),NaN,NaN,NaN,...,NaN,NaN,f,NaN,NaN,Coyhaique,Coyhaique,28763.0,29055.0,57818.0
85019,10039874,Coyhaique,2021-03-27,12,DESCARTADA,Atención médica,Hospital Regional (Coihaique),NaN,NaN,NaN,...,NaN,NO,f,2021-03-26,2021-03-29,Coyhaique,Coyhaique,28763.0,29055.0,57818.0
60630,7699027,Coyhaique,2021-02-01,5,DESCARTADA,Atención médica,Hospital Regional (Coihaique),NaN,NaN,NaN,...,NaN,DESCONOCIDO,f,NaN,2021-02-03,Coyhaique,Coyhaique,28763.0,29055.0,57818.0


In [15]:
df_seguimiento = pd.read_csv('data/20210831_Región de Aysén del General Carlos Ibáñez del Campo_Seguimiento.csv',
                            sep=';',
                             low_memory=False
                            )

In [16]:
df_seguimiento

,n_folio,tipo_seguimiento,dia_contacto,id_enfermedad_eno,enfermedad,fecha_notificacion,id_institucion,nombre_institucion_indice,region_institucion_indice,nombre_region_institucion_indice,...,folio_se_convierte_en_caso,cumple_aislamiento_cuarentena,ultimo_establecimiento_derivacion,nombre_ultimo_establecimiento_derivacion,ultimo_seremi_derivacion,región_derivación,región_seremi,región_notificacion,region_final,TTA
0,SC14618693-18497788,contacto,2021-07-08,50,Coronavirus (COVID-19),2021-07-05,13,Ejercicio libre de la profesión RM región,13,Región Metropolitana de Santiago,...,Sin Datos,t,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región de Aysén del General Carlos Ibáñez del ...,APS
1,14585350,caso,2021-07-12,50,Coronavirus (COVID-19),2021-07-05,200380,Centro de Salud Familiar Puerto Aysen,11,Región de Aysén del General Carlos Ibáñez del ...,...,Sin Datos,t,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,APS
2,14603344,caso,2021-07-06,50,Coronavirus (COVID-19),2021-07-05,125010,Actividades gestionadas por la Dirección del S...,11,Región de Aysén del General Carlos Ibáñez del ...,...,Sin Datos,t,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI
3,SC14574306-18460165,contacto,2021-07-13,50,Coronavirus (COVID-19),2021-07-05,125010,Actividades gestionadas por la Dirección del S...,11,Región de Aysén del General Carlos Ibáñez del ...,...,14899581,t,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI
4,SC14757148-18641978,contacto,2021-07-14,50,Coronavirus (COVID-19),2021-07-09,200380,Centro de Salud Familiar Puerto Aysen,11,Región de Aysén del General Carlos Ibáñez del ...,...,Sin Datos,t,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30959,SC16034389-20188901,contacto,2021-08-25,50,Coronavirus (COVID-19),2021-08-15,125100,Hospital Regional (Coihaique),11,Región de Aysén del General Carlos Ibáñez del ...,...,Sin Datos,t,125300,Consultorio Víctor Domingo Silva,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,APS
30960,PNTM-7016056768,no notificado índice,Sin Datos,50,Coronavirus (COVID-19),Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,...,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región de Aysén del General Carlos Ibáñez del ...,SEREMI
30961,PNTM-7015472903,no notificado índice,Sin Datos,50,Coronavirus (COVID-19),Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,...,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región de Aysén del General Carlos Ibáñez del ...,SEREMI
30962,PNTM-7015314831,no notificado índice,Sin Datos,50,Coronavirus (COVID-19),Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,...,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región de Aysén del General Carlos Ibáñez del ...,SEREMI


In [17]:
# Divide strings utilizando regex para extraer nfolio de casos, 
# Separa nfolio caso indice de nfolio contacto estrecho y guarda caso índice
# Y extrae datos de PNTM

df_seguimiento['n_folio'].str.extract(r'([0-9]+)|SC[0-9]+-([0-9]+)|PNTM-([0-9]+)', flags=0,expand=True).rename(columns={0:'nfolio_caso',
                                                                                                          1:'nfolio_caso_indice',
                                                                                                          2:'nfolio_pntm'
                                                                                                        })

,nfolio_caso,nfolio_caso_indice,nfolio_pntm
0,NaN,18497788,NaN
1,14585350,NaN,NaN
2,14603344,NaN,NaN
3,NaN,18460165,NaN
4,NaN,18641978,NaN
...,...,...,...
30959,NaN,20188901,NaN
30960,NaN,NaN,7016056768
30961,NaN,NaN,7015472903
30962,NaN,NaN,7015314831


In [18]:
# Separa nfolio caso índice de nfolio contacto estrecho y guarda contacto estrecho

df_seguimiento['n_folio'].str.extract(r'SC([0-9]+)-', 
                                      flags=0,
                                      expand=True).rename(
                                      columns={0:'nfolio_contacto_estrecho'})

,nfolio_contacto_estrecho
0,14618693
1,NaN
2,NaN
3,14574306
4,14757148
...,...
30959,16034389
30960,NaN
30961,NaN
30962,NaN


In [19]:
# Realiza merge con DF original utilizando index de importación
df_seguimiento_merged = df_seguimiento.merge(df_seguimiento['n_folio'].str.extract(
                                                 r'([0-9]+)|SC[0-9]+-([0-9]+)|PNTM-([0-9]+)', 
                                                 flags=0,
                                                 expand=True).rename(
                                                 columns={0:'nfolio_caso',
                                                          1:'nfolio_caso_indice',
                                                          2:'nfolio_pntm'}), 
                                             how='left',
                                             left_index= True,
                                             right_index=True
                                      ).merge(df_seguimiento['n_folio'].str.extract(
                                                                 r'SC([0-9]+)-', 
                                                                 flags=0,
                                                                 expand=True).rename(
                                                                 columns={0:'nfolio_contacto_estrecho'}), 
                                                                 how='left',
                                                                 left_index= True,
                                                                 right_index=True
                                      )

df_seguimiento_merged.sample(3)

,n_folio,tipo_seguimiento,dia_contacto,id_enfermedad_eno,enfermedad,fecha_notificacion,id_institucion,nombre_institucion_indice,region_institucion_indice,nombre_region_institucion_indice,...,ultimo_seremi_derivacion,región_derivación,región_seremi,región_notificacion,region_final,TTA,nfolio_caso,nfolio_caso_indice,nfolio_pntm,nfolio_contacto_estrecho
12670,15280149,caso,Sin Datos,50,Coronavirus (COVID-19),2021-07-24,125010,Actividades gestionadas por la Dirección del S...,11,Región de Aysén del General Carlos Ibáñez del ...,...,Sin Datos,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI,15280149,NaN,NaN,NaN
25822,SC15141131-19172775,contacto,2021-07-28,50,Coronavirus (COVID-19),2021-07-20,133170,Hospital Comunitario de Queilén,10,Región de Los Lagos,...,Sin Datos,Sin Datos,Sin Datos,Región De Los Lagos,Región de Aysén del General Carlos Ibáñez del ...,APS,NaN,19172775,NaN,15141131
2145,15668604,caso,Sin Datos,50,Coronavirus (COVID-19),2021-08-04,11,Ejercicio libre de la profesión XI región,11,Región de Aysén del General Carlos Ibáñez del ...,...,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región de Aysén del General Carlos Ibáñez del ...,SEREMI,15668604,NaN,NaN,NaN


In [20]:
# Revisa folio en DB de notificaciones

data_aysen_clean[data_aysen_clean['numero_folio']==14832232]

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,tipo_caso_busqueda,establecimiento_salud,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,...,hospitalizacion,trabajador_salud,caso_cerrado,fecha_primeros_sintomas,fecha_diagnostico,comuna_seremi_xi,provincia_seremi_xi,pop_hombres,pop_mujeres,pop_total
145734,14832232,Coyhaique,2021-07-11,28,CONFIRMADA,Atención médica,Hospital Regional (Coihaique),NEGATIVO,POSITIVO,POSITIVO,...,NaN,NO,f,2021-07-09,2021-07-12,Coyhaique,Coyhaique,28763.0,29055.0,57818.0


In [21]:
# Todos los datos poseen un folio 

df_seguimiento_merged[df_seguimiento_merged['n_folio'].isnull()]

,n_folio,tipo_seguimiento,dia_contacto,id_enfermedad_eno,enfermedad,fecha_notificacion,id_institucion,nombre_institucion_indice,region_institucion_indice,nombre_region_institucion_indice,...,ultimo_seremi_derivacion,región_derivación,región_seremi,región_notificacion,region_final,TTA,nfolio_caso,nfolio_caso_indice,nfolio_pntm,nfolio_contacto_estrecho


In [22]:
df_seguimiento_merged.columns

Index(['n_folio', 'tipo_seguimiento', 'dia_contacto', 'id_enfermedad_eno',
       'enfermedad', 'fecha_notificacion', 'id_institucion',
       'nombre_institucion_indice', 'region_institucion_indice',
       'nombre_region_institucion_indice', 'institucion_contacto',
       'nombre_institucion_seguimiento', 'region', 'id_region',
       'run_profesional', 'nombre_profesional', 'identificacion_paciente',
       'n_identificacion_paciente', 'nombre_paciente',
       'primer_apellido_paciente', 'fecha_cumple_requisitos',
       'cont_tipo_identificacion', 'cont_n_identificacion', 'cont_nombres',
       'cont_primer_apellido', 'cont_segundo_apellido', 'cont_sexo',
       'cont_fecha_nacimiento', 'cont_edad', 'cont_tipo_direccion',
       'cont_tipo_institucion', 'cont_via_residencia', 'cont_direccions',
       'cont_n_residencia', 'cont_dpto', 'cont_poblacion', 'cont_region',
       'cont_comuna', 'cont_n_telefono', 'cont_n_celular',
       'cont_correo_electronico', 'cont_fecha_seguimient

In [53]:
pd.DataFrame(df_seguimiento_merged.columns).rename(columns={0:'variables_seguimiento'}).transpose()

,0,1,2,3,4,5,6,7,8,9,...,78,79,80,81,82,83,84,85,86,87
variables_seguimiento,n_folio,tipo_seguimiento,dia_contacto,id_enfermedad_eno,enfermedad,fecha_notificacion,id_institucion,nombre_institucion_indice,region_institucion_indice,nombre_region_institucion_indice,...,ultimo_seremi_derivacion,región_derivación,región_seremi,región_notificacion,region_final,TTA,nfolio_caso,nfolio_caso_indice,nfolio_pntm,nfolio_contacto_estrecho


In [27]:
df_seguimiento_merged['folio_se_convierte_en_caso']

0        Sin Datos
1        Sin Datos
2        Sin Datos
3         14899581
4        Sin Datos
           ...    
30959    Sin Datos
30960    Sin Datos
30961    Sin Datos
30962    Sin Datos
30963     16161119
Name: folio_se_convierte_en_caso, Length: 30964, dtype: object

In [116]:
# Revisa existencia de una variable dentro del DF y sus respectivos valores

df_seguimiento_merged['folio_se_convierte_en_caso'].value_counts()


Sin Datos    30277
-1              20
15343397        14
14560708        12
14951542        11
             ...  
15170161         1
15745320         1
15143562         1
16161119         1
16074522         1
Name: folio_se_convierte_en_caso, Length: 184, dtype: int64

# Variables de interes


|Variable|Tipo|Observación|
|---|---|---|
|`n_folio`|Text|folio generado por Número y Letras|
|`nfolio_caso`|Numeric|folio de caso registrado|
|`nfolio_caso_indice`|Numeric|folio de caso indice registrado|
|`nfolio_pntm`|Numeric|folio plataforma nacional de toma de muestra|
|`nfolio_contacto_estrecho`|Numeric|folio de contacto estrecho|
|`tipo_seguimiento`|Text|contacto, caso, no notificado indice|
|`dia_contacto`|DateTime|Fecha de contacto |
|`fecha_notificacion`|DateTime|Fecha de notificación|
|`nombre_institucion_indice`|Text|Nombre institución desde donde se deriva el caso|
|`region_institucion_indice`|Numeric| Región d ela cual se deriva el caso|
|`nombre_region_institucion_indice`|Text|Nombre de región de la institución índice|
|`nombre_institucion_seguimiento`|Text|Nombre de la institución encargada de realizar seguimiento|
|`region`|Text|Región en la cual se realiza el seguimiento|
|`id_region`|Numeric|Número de la región de seguimiento|
|`fecha_cumple_requisitos`|DateTime|Fecha en la cual cumple requisitos para seguimiento|
|`TTA`|Text|APS o SEREMI|
|`cumple_aislamiento_cuarentena`|Boolean|Sin datos, t, f|
|`folio_se_convierte_en_caso`|Text|Número de folio o Sin Datos|
|`visible_no_eliminado`|Boolean|Sin datos, t, f|
|`suspendido`|Text|egresado|
|`sintomas`|Text|Sintomatología, Sin datos o Ninguno|
|`observacion_seguimiento_contactos`|Text| Escrito por el trazador|
|`tiene_resultado_covid`|Text|Positivo, Negativo, No concluyente, Sin Datos|
|`derivacion_a_su`|Boolean| t f Sin datos|
|`derivacion_toma_muestra`|Boolean|t f Sin datos|
|`fecha_derivacion_toma_muestra`|Text|Fecha Sin datos|
|`cont_fecha_seguimiento`|Text|Texto o Sin Datos|
|`cont_tipo_contacto`|Text|contacto_familiar, contacto_social, contacto_institucional, contacto_laboral|
|||contacto_pasajero, contacto_sala_espera, contacto_personal_salud,Datos|
|`cont_parentesco`|Text|Hijo/a, Madre/Padre, Otro familiar, Pareja, Hermano/a, Otra relación/No familiar, Sin Datos|
|`cont_otra_relacion`|Text|Sin Datos, otro, hijo de cuidadora, otro adulto mayor en cuidado, cuñado. sobrino de cuidadora, AMIGO,  Amigo|
||| pareja del tío de su madre,cuidadora, PAREJA PADRE, otro adulto en cuidados, Pareja de mamá, Hijastro|
|`cont_tipo_transporte`|Text|Aéreo, Sin Datos, Terrestre|
|`cont_nombre_vuelo`|Text|Sin Datos, JETSMART JA245 CCP - BALMACEDA, SKY 433, COPA CM111, 
|||SKY 433 STGO- COYHAIQUE 21-08-2021 / 11 AM, SK, DL147 ATLANTA - STGO, JETSMART CCP-BALMACEDA, LA 272 |
|||JETSMART JA245, VUELO CM111, LA272, JETSMART JA 245 CCP - BALMACEDA|                 
|`cont_fecha_llegada`|Text|Sin Datos, 2021-07-21, 2021-08-21, 2021-07-01, 2021-07-03, 2021-07-09, 2021-07-02|
|`detalle_pasajero`|Text|Observaciones adicionales de viaje|
|`cont_inicio_cuarentena`|DateTime|Fecha de inicio de cuarentena|
|`cont_fin_cuarentena`|DateTime|Fecha y Sin Datos|
|`cont_requiere_licencia_medica`|Boolean|Sin Datos, NO, SI, Seleccione|
|`cont_se_emite_licencia`|Boolean|Sin Datos, t, f|
|`cont_fecha_licencia`|DateTime|Fecha de licencia|
|`requiere_canasta_alimentos`|Boolean|Sin Datos, t, f|
|`cantidad_personas_vivienda`|Numeric|Integrantes familiares de 0 a 6|
|`cont_estado_seguimiento`|Text|Aún no contactado, En proceso de seguimiento, Se transforma en caso, Contacto sintomático, No localizado|
|`fecha_creacion`|DateTime|Fecha de creación|
|`tipo_del_contacto`|Text|Sin Datos, Llamada telefónica, Visita domiciliaria|
|`contacto_localizado`|Boolean|Sin Datos, t, f|
|`empresa`|Text|Información de Empresa de trabajo|
|`institucion`|Text|Información de la institución|

## Limpieza y ordenamiento de datos



Segun variables seleccionadas como de importancia de análisis y obsrvaciones a filtrar

In [100]:
# Transforma fecha_cumple_requisitos a formato DateTime64 

df_seguimiento_merged['fecha_cumple_requisitos'] = pd.to_datetime(df_seguimiento_merged['fecha_cumple_requisitos'], errors='coerce')
df_seguimiento_merged['fecha_notificacion'] = pd.to_datetime(df_seguimiento_merged['fecha_notificacion'], errors='coerce')
df_seguimiento_merged['fecha_creacion'] = pd.to_datetime(df_seguimiento_merged['fecha_creacion'], errors='coerce')

#Crea diferencia de fecha
df_seguimiento_merged['dif_fecha'] = df_seguimiento_merged['fecha_cumple_requisitos'] - df_seguimiento_merged['fecha_notificacion']

In [101]:
# Filtra los timedelta mayores a 14 dias

seg = df_seguimiento_merged[df_seguimiento_merged['dif_fecha']<pd.to_timedelta('14 days')]

In [102]:
#Filtra visible_no_eliminado y bota duplicados en n_folio

aux = seg[seg['visible_no_eliminado']=='t'].drop_duplicates('n_folio')

In [103]:
# Filtra tipo_seguimiento para los casos 

aux_caso = aux[aux['tipo_seguimiento']=='caso']

In [110]:
aux_contacto = aux[aux['tipo_seguimiento']=='contacto']
#Crea columna dif_fecha_contacto
aux_contacto['dif_fecha_contacto'] = aux_contacto['fecha_creacion']-aux_contacto['fecha_notificacion']

aux_contacto = aux_contacto[aux_contacto['dif_fecha_contacto']>=pd.to_timedelta(-14)]

<ipython-input-110-483468821897>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux_contacto['dif_fecha_contacto'] = aux_contacto['fecha_creacion']-aux_contacto['fecha_notificacion']


In [113]:
aux_contacto_nuevos = aux[aux['tipo_seguimiento']=='contacto']
aux_contacto_nuevos = aux_contacto_nuevos[(aux_contacto_nuevos['suspendido']=='activo')|(aux_contacto_nuevos['suspendido']=='egresado')]


In [ ]:
aux_contacto_viejos = aux_contacto[aux_contacto[]]
aux_contacto_viejos= aux_contacto_viejos[(aux_contacto_viejos['suspendido']=='activo')|(aux_contacto_viejos['suspendido']=='egresado')]


In [ ]:
df_seguimiento_merged['visible_no_eliminado']

In [ ]:
df_seguimiento_filtered = df_seguimiento_merged

In [59]:
# Selecciona variables de importancia

df_seguimiento_clean = df_seguimiento_filtered[[
    'n_folio',
    'nfolio_caso',
    'nfolio_caso_indice',
    'nfolio_pntm',
    'nfolio_contacto_estrecho',
    'tipo_seguimiento',
    'dia_contacto',
    'fecha_notificacion',
    'nombre_institucion_indice',
    'region_institucion_indice',
    'nombre_region_institucion_indice',
    'region',
    'id_region',
    'fecha_cumple_requisitos',
    'TTA',
    'cumple_aislamiento_cuarentena',
    'folio_se_convierte_en_caso',
    'visible_no_eliminado',
    'suspendido',
    'sintomas',
    'observacion_seguimiento_contactos',
    'tiene_resultado_covid',
    'derivacion_a_su',
    'derivacion_toma_muestra',
    'fecha_derivacion_toma_muestra',
    'cont_fecha_seguimiento',
    'cont_tipo_contacto',
    'cont_parentesco',
    'cont_otra_relacion',
    'cont_tipo_transporte',
    'cont_nombre_vuelo',
    'cont_fecha_llegada',
    'detalle_pasajero',
    'cont_inicio_cuarentena',
    'cont_fin_cuarentena',
    'cont_requiere_licencia_medica',
    'cont_se_emite_licencia',
    'cont_fecha_licencia',
    'requiere_canasta_alimentos',
    'cantidad_personas_vivienda',
    'cont_estado_seguimiento',
    'fecha_creacion',
    'tipo_del_contacto',
    'contacto_localizado',
    'empresa',
    'institucion'
]]

In [60]:
df_seguimiento_clean

,n_folio,nfolio_caso,nfolio_caso_indice,nfolio_pntm,nfolio_contacto_estrecho,tipo_seguimiento,dia_contacto,fecha_notificacion,nombre_institucion_indice,region_institucion_indice,...,cont_se_emite_licencia,cont_fecha_licencia,requiere_canasta_alimentos,cantidad_personas_vivienda,cont_estado_seguimiento,fecha_creacion,tipo_del_contacto,contacto_localizado,empresa,institucion
0,SC14618693-18497788,NaN,18497788,NaN,14618693,contacto,2021-07-08,2021-07-05,Ejercicio libre de la profesión RM región,13,...,Sin Datos,Sin Datos,Sin Datos,Sin Datos,En proceso de seguimiento,2021-07-07 12:18:26.894552,Llamada telefónica,t,NaN,NaN
1,14585350,14585350,NaN,NaN,NaN,caso,2021-07-12,2021-07-05,Centro de Salud Familiar Puerto Aysen,11,...,Sin Datos,Sin Datos,f,Sin Datos,En proceso de seguimiento,2021-07-05 12:32:39.304674,Llamada telefónica,t,NaN,NaN
2,14603344,14603344,NaN,NaN,NaN,caso,2021-07-06,2021-07-05,Actividades gestionadas por la Dirección del S...,11,...,Sin Datos,Sin Datos,f,Sin Datos,En proceso de seguimiento,2021-07-05 16:32:13.222828,Llamada telefónica,t,NaN,NaN
3,SC14574306-18460165,NaN,18460165,NaN,14574306,contacto,2021-07-13,2021-07-05,Actividades gestionadas por la Dirección del S...,11,...,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Contacto sintomático,2021-07-06 16:27:11.452588,Llamada telefónica,t,NaN,NaN
4,SC14757148-18641978,NaN,18641978,NaN,14757148,contacto,2021-07-14,2021-07-09,Centro de Salud Familiar Puerto Aysen,11,...,Sin Datos,Sin Datos,Sin Datos,Sin Datos,En proceso de seguimiento,2021-07-09 21:24:23.366254,Llamada telefónica,t,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30959,SC16034389-20188901,NaN,20188901,NaN,16034389,contacto,2021-08-25,2021-08-15,Hospital Regional (Coihaique),11,...,Sin Datos,Sin Datos,Sin Datos,Sin Datos,En proceso de seguimiento,2021-08-17 12:10:55.321584,Llamada telefónica,t,NaN,Escuela nieves del sur
30960,PNTM-7016056768,NaN,NaN,7016056768,NaN,no notificado índice,Sin Datos,Sin Datos,Sin Datos,Sin Datos,...,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Aún no contactado,2021-08-16 00:32:44.242543,Sin Datos,Sin Datos,Sin Datos,Sin Datos
30961,PNTM-7015472903,NaN,NaN,7015472903,NaN,no notificado índice,Sin Datos,Sin Datos,Sin Datos,Sin Datos,...,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Aún no contactado,2021-08-05 00:32:25.801614,Sin Datos,Sin Datos,Sin Datos,Sin Datos
30962,PNTM-7015314831,NaN,NaN,7015314831,NaN,no notificado índice,Sin Datos,Sin Datos,Sin Datos,Sin Datos,...,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Aún no contactado,2021-08-01 00:32:32.553682,Sin Datos,Sin Datos,Sin Datos,Sin Datos


# Indicador 5: Oportunidad en la investigación epidemiológica de casos

## Objetivo: Fortalecer la investigación epidemiológica de los casos

### Meta: >90%

**Periodicidad:** Semanal

**Escala Territorial:** Comunal, Regional, Nacional

**Fuente de Información:** Base EPIVIGILIA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:casos\:nuevos\:registrados\:investigados\:dentro\:de\:48\:hrs\:por\:comuna\:1er\:establecimiento,\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:comuna\:1er\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:casos\:nuevos\:registrados\:investigados\:dentro\:de\:48\:hrs\:por\:región\:1er\:establecimiento,\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:región\:1er\:establecimiento\:semana\:x}x100$

**Observación**

- No se incluyen los casos nuevos no notificados
- Se considera un caso nuevo registrado a aquel caso notificado que cumple los requisitos para estar en el modulo de seguimiento
- La fecha para determinar el inicio de seguimiento corresponde a la fecha de resultado (caso confirmado) o fecha de notificación (caso probable). Para un caso confirmado, si el resultado del test es anterior a la fecha de notificación, se debe iniciar seguimiento a partir de la fecha en que el registro está disponible en la bandeja de seguimiento.
- Se cumple el indicador si, dentro de las 48 hrs. a partir de la fecha de inicio de seguimiento se tiene al menos una actividad de seguimiento exitosa (se logra ubicar al caso).
- El primer establecimiento corresponde al establecimiento dado por API FONASA. Si el registro no hace match con API FONASA, se considera el establecimiento de notificación.
- Se consideran sólo registros de casos que estén activos o egresados en la bandeja de seguimiento.

# Indicador 6: Capacidad de investigación de contactos

## Objetivos: Fortalecer la investigación epidemiológica de los contactos estrechos

### Meta: >80%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

**Cálculo del indicador a nivel comunal**

$\frac{No.\:casos\:nuevos\:con\:contactos\:estrechos\:por\:comuna\:1er\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:por\:comuna\:1er\:establecimiento\:semana\:x}$

**Cálculo del indicador a nivel regional**

$\frac{No.\:casos\:nuevos\:con\:contactos\:estrechos\:por\:región\:1er\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:por\:región\:1er\:establecimiento\:semana\:x}$

**Observaciones**

- No se incluyen los casos nuevos no notificados
- Se considera un caso nuevo registrado a aquel caso nuevo notificado que cumple los requisitos para estar en el módulo de seguimiento
- Los casos nuevos con contactos estrechos son considerados de forma dicotómica (sí o no)
- El primer establecimiento corresponde al establecimiento de notificación. Se seguirá considerando el primer establecimiento para este indicador, aún si se derivó previo a la investigación epidemiológica.
- Se consideran sólo registros activos o egresados en la bandeja de seguimiento

# Indicador 7: Capacidad de investigación de contactos

## Objetivo: Evaluar la capacidad de seguimiento de contactos

### Meta: >80%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento.

**Cálculo del indicador a nivel comunal**

$\frac{No.\:contactos\:identificados\:investigados\:dentro\:de\:48hrs\:por\:comuna\:1er\:establecimiento\:semana\:x}{No.\:contactos\:identificados\:por\:comuna\:1er\:establecimiento\:semana\:x}x100$

**Cálculo del indicador a nivel regional**

$\frac{No.\:contactos\:identificados\:investigados\:dentro\:de\:48hrs\:por\:región\:1er\:establecimiento\:semana\:x}{No.\:contactos\:identificados\:por\:región\:1er\:establecimiento\:semana\:x}x100$

**Observaciones**

- No se incluyen los contactos nuevos no notificados
- Se considera como registro de contacto aquel que cumple las condiciones para estar en la bandeja de seguimiento
- Se consideran sólo registros de casos activos o egresados en la bandeja de seguimiento
- El primer establecimiento corresponde al establecimiento dado por API FONASA. Si el registro no hace match con API FONASA, se considera el establecimiento para este indicador, aún si se derivó previo a la investigación epidemiológica

# Indicador 8: Casos en seguimiento

## Objetivo: Evaluar la capacidad de seguimiento de casos

### Meta: >90%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de Información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:casos\:activos\:investigados\:con\:seguimiento\:según\:protocolo\:por\:comuna\:último\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:comuna\:ultimo\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:casos\:activos\:investigados\:con\:seguimiento\:según\:protocolo\:por\:región\:último\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:región\:ultimo\:establecimiento\:semana\:x}x100$

**Observación**

- No se incluyen los casos nuevos no notificados ("casos reportados por laboratorio")
- Se considera un caso activo como aquel caso notificado que cumple la definición de caso activo, con fecha de notificación en la semana de interés
- Se consideran sólo registros activos o egresados en la bandeja de seguimiento
- Según protocolo implica un seguimiento diario por día de seguimineto, desde el inicio de seguimiento y se valora de forma dicotómica (cumple o no cumple el protocolo de seguimiento)
- Por seguimiento diario se entiende que diariamente, se debe realizar una actividad de seguimiento (la cual puede ser exitosa o no)

# Indicador 10: Contactos con al menos 2 seguimientos dentro de su periodo de cuarentena de 14 días

## Objetivo: Evaluar la capacidad de seguimiento de contactos estrechos

### Meta: >90%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:contactos\:que\:cumplieron\:cuarentena\:con\:al\:menos\:2\:seguimientos\:por\:comuna\:último\:establecimiento\:semana\:x}{No.\:contactos\:que\:cumplieron\:cuarentena\:por\:comuna\:ultimo\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:contactos\:que\:cumplieron\:cuarentena\:con\:al\:menos\:2\:seguimientos\:por\:región\:último\:establecimiento\:semana\:x}{No.\:contactos\:que\:cumplieron\:cuarentena\:por\:región\:ultimo\:establecimiento\:semana\:x}x100$

**Observación**

- El indicador se construye sobre los contactos que egresan (fecha de término de cuarentena) de seguimiento, durante la semana de interés (esto es, sólo registros egresados de la bandeja de seguimiento)
- El periodo de seguimiento para el contacto corresponde a la cantidad de días entre la 1era fecha en que el contacto está disponible en la bandeja de seguimiento hasta la última fecha fin cuarentena
- Se cumple con el indicador si en el período de seguimiento del contacto se registran al menos 2 actividades de seguimiento (las cuales pueden ser exitosas o no)
- Se excluyen los contactos que se transforman en caso y que están notificados (tiene folio asociado como caso)